In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy import distance
from geopy.extra.rate_limiter import RateLimiter
import json
import glob
import os
import shapefile as shp
from difflib import SequenceMatcher
geolocator=Nominatim(user_agent="bike_application")
trips11=pd.read_csv(r"C:\Users\giorg\Desktop\Trips\capitalbikeshare_tripdata_11.csv")
files=os.path.join("C:\\Users\\giorg\\Desktop\\Trips\\","capitalbikeshare_tripdata_12q*.csv")
files=glob.glob(files)
trips12=pd.concat(map(pd.read_csv,files),ignore_index=True)
trips11.columns=trips11.columns.str.replace(" ","_").str.lower()
trips12.columns=trips12.columns.str.replace(" ","_").str.lower()
trips11["start_date"]=pd.to_datetime(trips11["start_date"])
trips11["end_date"]=pd.to_datetime(trips11["end_date"])
trips11["duration"]=(trips11["duration"]/60).round(1)
trips12["start_date"]=pd.to_datetime(trips12["start_date"])
trips12["end_date"]=pd.to_datetime(trips12["end_date"])
trips12["duration"]=(trips12["duration"]/60).round(1)
stations11=trips11["start_station"].unique()
stations11=pd.DataFrame(stations11,columns=["station"])
stations11["lat"]=np.nan
stations11["long"]=np.nan

In [1]:
import geopandas

PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


In [70]:
with open("en_station_information.json","r") as json_file:
    json_load=json.load(json_file)
station_info=pd.DataFrame(json_load["data"]["stations"])
station_info=station_info.drop(["electric_bike_surcharge_waiver","eightd_station_services","station_type","rental_uris","rental_methods",
"external_id","has_kiosk","eightd_has_key_dispenser"],axis=1)

In [71]:
station_info["region_id"]=station_info["region_id"].fillna(0)
station_info["region_id"]=station_info["region_id"].astype("int64")
station_info["region"]=station_info["region_id"].map({40:"Alexandria Virginia",41:"Arlington Virginia",42:"Washington DC",43:"Montgomery County Maryland",
44:"Montgomery Country Maryland",48:"Test and Operations",104:"Fairfax Virginia",128:"8D",133:"Prince George\'s County Maryland",152:"Falls Church Virginia"}).astype("category")

In [72]:
for i in range(len(stations11)):
    for j in range(len(station_info)):
        if stations11.loc[i,"station"]==station_info.loc[j,"name"]:
            stations11.loc[i,"lat"]=station_info.loc[j,"lat"]
            stations11.loc[i,"long"]=station_info.loc[j,"lon"]


In [73]:
stations11=stations11.fillna(0)
stations11["region"]=0

In [81]:
#We see that there are slight differences in the adresses spelling, so we consider adresses with more than 80% similarity to be the same
for i in range(len(stations11)):
    for j in range(len(station_info)):
        if (SequenceMatcher(None, stations11.loc[i,"station"],station_info.loc[j,"name"])).ratio()>0.8:
            stations11.loc[i,"region"]=" "
            stations11.loc[i,"region"]=stations11.loc[i,"region"]+station_info.loc[j,"region"]

In [82]:
#Now we find the coordinates for those adresses
geocode=RateLimiter(geolocator.geocode, min_delay_seconds=1)
for i in range(len(stations11)):
    if stations11.loc[i,"lat"]==0 or stations11.loc[i,"long"]==0:
        if stations11.loc[i,"region"]==0:
            a=geocode(stations11.loc[i,"station"],timeout=None)    
        else:
            a=geocode((stations11.loc[i,"station"]+stations11.loc[i,"region"]),timeout=None)
        try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations11.loc[i,"lat"]=la
                stations11.loc[i,"long"]=lo
        except AttributeError:
            pass

In [83]:
for i in range(len(stations11)):
    if stations11.loc[i,"lat"]==0 or stations11.loc[i,"long"]==0:
        if "/" in stations11.loc[i,"station"]:
            j,x=((stations11.loc[i,"station"]).split("/"))
            a=geocode(x)
        else:
            a=geocode(stations11.loc[i,"station"])
    try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations11.loc[i,"lat"]=la
                stations11.loc[i,"long"]=lo
    except AttributeError:
            pass
for i in range(len(stations11)):
    if stations11.loc[i,"lat"]==0 or stations11.loc[i,"long"]==0:
        a=geocode(stations11.loc[i,"station"])
        try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations11.loc[i,"lat"]=la
                stations11.loc[i,"long"]=lo
        except AttributeError:
            pass

In [84]:
trips11["lat_start"]=0
trips11["long_start"]=0
trips11["lat_end"]=0
trips11["long_end"]=0

In [85]:
#Takes centuries to run, use it only during night
# for i in range(len(trips11)):
#     for j in range(len(stations11)):
#         if trips11.loc[i,"start_station"]==stations11.loc[j,"station"]:
#             trips11.loc[i,"lat_start"]=stations11.loc[j,"lat"]
#             trips11.loc[i,"long_start"]=stations11.loc[j,"long"]
#         if trips11.loc[i,"end_station"]==stations11.loc[j,"station"]:
#             trips11.loc[i,"lat_end"]=stations11.loc[j,"lat"]
#             trips11.loc[i,"long_end"]=stations11.loc[j,"long"]
            

In [91]:
i=stations11.loc[stations11["lat"]!=0]
j=stations11.loc[stations11["long"]!=0]
bbox=(i["lat"].max(),i["lat"].min(),j["long"].max(),j["long"].min())

In [97]:
38.947774, -76.941877, 38.8425997, -77.087171

(38.947774, 38.8425997, -76.941877, -77.087171)